In [ ]:
#%load_ext autoreload
#%autoreload 2

#%pip install magic_duckdb --upgrade --quiet
%load_ext magic_duckdb

%dql -t show pragma version

types = %dql --listtypes
print(types)

In [ ]:
# List available functions
%dql -d

%dql -t show pragma version 

In [ ]:
# Line magic with PyArrow
%dql -t arrow

%dql CREATE OR REPLACE TABLE xyz as SELECT * FROM range(100) t(x)

table1 = %dql SELECT * FROM xyz where x>50

display(table1)

In [ ]:
# List tables used by a query
%dql --tables with abc as (select * from def) select * from abc, tbl1, tbl3


In [ ]:
# Line magic with Pandas DataFrame
%dql -t df
%dql CREATE OR REPLACE TABLE xyz as SELECT * FROM range(100) t(x)

df1 = %dql SELECT * FROM xyz where x>50

display(len(df1))

In [ ]:
%%dql

CREATE OR REPLACE TABLE abc as SELECT * FROM range(200) c(a) where a % 7 == 0;
CREATE OR REPLACE TABLE def as SELECT * FROM range(200) c(a) where a % 2 == 0;

SELECT count(*) FROM def join abc on def.a=abc.a;

In [ ]:
# Get the connection created within DQL and use it directly
con = %dql --getcon
display(con.sql("pragma version").df())

In [ ]:
# Use an externally created connection
import duckdb

dql_test_con = duckdb.connect("file1919.db")
dql_test_con.sql("CREATE OR REPLACE TABLE xyz as select * from range(100)")

# -co sets the connection to an existing object
%dql -co dql_test_con
%dql select count(*) from xyz

In [ ]:
# Pass a connection string via dql -cn
%dql -cn file19192.db
%dql create or replace table xyz as select * from range(1);select count(*) from xyz

In [ ]:
# Pass a connection string via dql -cn, in this case, :memory: which is the same as using the -d default connection
%dql -cn :memory:
%dql pragma version

In [ ]:
# Combining DQL and a relationship

%dql -t relation
r = %dql select * from (select * from range(105) t(x)) join (select * from range(10)) on true
r.filter('x > 60').show()

In [ ]:
# Simple Explain

r = %dql -e explain select * from (select * from range(105) t(x)) join (select * from range(10)) on true
print(r)

r = %dql -e explain_analyze_tree select * from (select * from range(105) t(x)) join (select * from range(10)) on true
print(r)

r = %dql -e explain_analyze_json select * from (select * from range(105) t(x)) join (select * from range(10)) on true
print(r)

# Simple Explain


In [ ]:
%%dql -t df 
-- Cell Magic with an Option
select * from (select * from range(105) t(x)) join (select * from range(10)) on true


In [ ]:
%dql -t df select * from (select * from range(105) t(x)) join (select * from range(10)) on true
# Line Magic with an Option


In [ ]:
# Get the last variable. This is one way to get the output of a cell and line magic. Another way is: obj = %dql ... 
r = _
print(r)

In [ ]:
# Miscellaneous examples

# Set the default_connection
duckdb.default_connection = duckdb.connect(":memory:") 
# DQL will use the default connection. 
%dql -d  
# returns the dql connection object
con = %dql --getcon 
%dql -t df
# stores result of pragma version in object named myversion
%dql -o myversion pragma version 
display(myversion)

In [ ]:
# Experimental SQL Formatting using https://github.com/sql-formatter-org/sql-formatter. 
# To use this feature, first run: 
# npm install sql-formatter
# Executes "npx sql-formatter" via popen

%dql --format select * from blah


In [ ]:
# define a var, use -r to replace the variables inside the magic, then confirm it worked

table1 = "mytableisawesome"
%dql -t df -r create or replace table {table1} as select * from range(100)
%dql select * from mytableisawesome

In [ ]:
# List tables

table1 = "mytableisawesome"
%dql -t df -r create or replace table {table1} as select * from range(100)
%dql select * from mytableisawesome

In [ ]:
# Draw the Explain Analyze

query = "select * from (select * from range(105) t(x)) join (select * from range(10)) on true"

try:
    import magic_duckdb
    # magic_duckdb.extras.explain_analyze_graphviz.dot_path= "c:\\Program files\\graphviz\\bin\\dot.exe"
    import graphviz
    graphviz.set_jupyter_format('png')

    mode="explain_analyze_draw"
    r = %dql -r -e {mode} {query}
    display(r)
except Exception as e:
    print("Graphviz not available, using explain_analyze_tree")
    mode="explain_analyze_tree"
    r = %dql -r -e {mode} {query}
    print(r)

